In [14]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [15]:
%matplotlib inline
labels = pd.read_csv("validation.csv", delimiter = ";")

In [16]:
data_train = df.iloc[:,1:]
data_test= test.iloc[:,1:]
data_labels = labels

testdf= pd.read_csv("best_result_so_far.csv", sep=";")
print data_test.shape
print testdf.shape

labels_test=testdf.values


(984, 3728)
(984, 96)


In [17]:

PM2 = data_labels.filter(regex='PM2')
PM10 = data_labels.filter(regex='PM10')
O3 = data_labels.filter(regex= "O3")
NO2= data_labels.filter(regex= "NO2")

n = data_test.shape[0]
m = data_labels.shape[1]

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb 
from time import time


#y_pred = np.zeros((n,m))
i=0
s = [PM2, PM10, O3,NO2]
clfs = []

params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 5

plst = list(params.items())
xgtest = xgb.DMatrix(data_test)

num_rounds = 2000
for target in s:
    
    l = len(target.columns)
    col = target.columns[0]

    print col
    xgtrain = xgb.DMatrix(data_train, label=target[col])
    
    
    start = time()
    model = xgb.train(plst, xgtrain, num_rounds)  
    end = time() - start
    print "Fitting the model took %d seconds"%end

    pred = model.predict(xgtest)
    labels_test[:, i] = pred[:]
    i= i+24

In [11]:
from postprocessing import *
y_filtered = multi_polutant_averaged_postprocessing(labels_test, method = "Selection", weights=None)
y_pred_final = pd.DataFrame(y_filtered, columns = data_labels.columns)
y_pred_final.to_csv("./result_filtered.csv",index=False,sep=";")


MSE :  8.43513213464
R2 :  0.936010335225
